**분석용데이터구축_황유정**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings(action="ignore")


In [ ]:
data=pd.read_csv('/kaggle/input/ccdata/CC GENERAL.csv')
data.head(50)

- CUSTID : Identification of Credit Card holder (Categorical)
- BALANCE : Balance amount left in their account to make purchases (
- BALANCEFREQUENCY : How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
- PURCHASES : Amount of purchases made from account
- ONEOFFPURCHASES : Maximum purchase amount done in one-go
- INSTALLMENTSPURCHASES : Amount of purchase done in installment
- CASHADVANCE : Cash in advance given by the user
- PURCHASESFREQUENCY : How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
- ONEOFFPURCHASESFREQUENCY : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
- PURCHASESINSTALLMENTSFREQUENCY : How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- CASHADVANCEFREQUENCY : How frequently the cash in advance being paid
- CASHADVANCETRX : Number of Transactions made with "Cash in Advanced"
- PURCHASESTRX : Numbe of purchase transactions made
- CREDITLIMIT : Limit of Credit Card for user
- PAYMENTS : Amount of Payment done by user
- MINIMUM_PAYMENTS : Minimum amount of payments made by user
- PRCFULLPAYMENT : Percent of full payment paid by user
- TENURE : Tenure of credit card service for user

In [ ]:
data.isnull().sum()

In [ ]:
# fill null with the mean of each column
data.loc[(data['MINIMUM_PAYMENTS'].isnull()==True),'MINIMUM_PAYMENTS']=data['MINIMUM_PAYMENTS'].mean()
data.loc[(data['CREDIT_LIMIT'].isnull()==True),'CREDIT_LIMIT']=data['CREDIT_LIMIT'].mean()

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
# Dealing with outliers
# based on the mean of each column

columns=['BALANCE','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS']

for c in columns:
    RANGE=c+'_RANGE'
    data[RANGE]=0
    data.loc[((data[c]>0) & (data[c]<500)), RANGE]=1
    data.loc[((data[c]>=500) & (data[c]<1000)), RANGE]=2
    data.loc[((data[c]>=1000) & (data[c]<3000)), RANGE]=3
    data.loc[((data[c]>=3000) & (data[c]<5000)), RANGE]=4
    data.loc[((data[c]>=5000) & (data[c]<10000)), RANGE]=5
    data.loc[(data[c]>=10000), RANGE]=6

In [ ]:
columns=['BALANCE_FREQUENCY', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY','CASH_ADVANCE_FREQUENCY','PRC_FULL_PAYMENT']

for c in columns:
    RANGE= c + '_RANGE'
    data[RANGE]=0
    data.loc[((data[c]>0)&(data[c]<0.1)), RANGE]=1
    data.loc[((data[c]>=0.1)&(data[c]<0.2)), RANGE]=2
    data.loc[((data[c]>=0.2)&(data[c]<0.3)), RANGE]=3
    data.loc[((data[c]>=0.3)&(data[c]<0.4)), RANGE]=4
    data.loc[((data[c]>=0.4)&(data[c]<0.5)), RANGE]=5
    data.loc[((data[c]>=0.5)&(data[c]<0.6)), RANGE]=6
    data.loc[((data[c]>=0.6)&(data[c]<0.7)), RANGE]=7
    data.loc[((data[c]>=0.7)&(data[c]<0.8)), RANGE]=8
    data.loc[((data[c]>=0.8)&(data[c]<0.9)), RANGE]=9
    data.loc[((data[c]>=0.9)&(data[c]<1.0)), RANGE]=10
    data.loc[(data[c]>=1.0), RANGE]=11
    

In [ ]:
columns=['CASH_ADVANCE_TRX','PURCHASES_TRX']

for c in columns:
    RANGE= c+'_RANGE'
    data[RANGE]=0
    data.loc[((data[c]>0)&(data[c]<=5)),RANGE]=1
    data.loc[((data[c]>5)&(data[c]<=10)),RANGE]=2
    data.loc[((data[c]>10)&(data[c]<=15)),RANGE]=3
    data.loc[((data[c]>15)&(data[c]<=20)),RANGE]=4
    data.loc[((data[c]>20)&(data[c]<=30)),RANGE]=5
    data.loc[((data[c]>30)&(data[c]<=50)),RANGE]=6
    data.loc[((data[c]>50)&(data[c]<=100)),RANGE]=7
    data.loc[((data[c]>100)),RANGE]=8

In [ ]:
data.drop(['BALANCE','BALANCE_FREQUENCY','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE','PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY',
          'CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS','PRC_FULL_PAYMENT'], axis=1, inplace=True)

In [ ]:
data.head()

x=np.array(data.drop(['CUST_ID'], axis=1))

In [ ]:
# Normalizing input data
scale=StandardScaler()
x=scale.fit_transform(x)
x.shape

In [ ]:
# Modeling using KMeans clustering
# Elbow Method

distortion=[]
iterations=3000
num_centroid_seeds=10
rand_state=0

for i in range(1, 30):
    kmeans=KMeans(n_clusters=i, max_iter=iterations, n_init=num_centroid_seeds, random_state=rand_state)
    kmeans.fit(x)
    distortion.append(kmeans.inertia_)

    
plt.plot(range(1,30), distortion, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('distortion')
plt.show()

In [ ]:
# number of cluster=7
kmeans=KMeans(n_clusters=7, max_iter=iterations, n_init=num_centroid_seeds, random_state=rand_state)
kmeans_preds=kmeans.fit_predict(x)
kmeans_preds # predict included class 
labels=kmeans.labels_
labels

In [ ]:
clusters=pd.concat([data, pd.DataFrame({'cluster':labels})], axis=1)
clusters.head()

In [ ]:
# scores of clustering
from sklearn.metrics import silhouette_samples, silhouette_score
print('silhouette_score:',silhouette_score(x, labels=kmeans.labels_))
print('K-Means centers:', kmeans.cluster_centers_)
print('k-Menas labels:', kmeans.labels_)

In [ ]:
# interpretation of Clusters
for c in clusters:
    grid=sns.FacetGrid(clusters, col='cluster')
    grid.map(plt.hist, c)

- cluster0: 조금벌고 조금사는 한번갈때 조금 삼, 할부금액 매우적음/할부 빈도수 적음, 현금인출 x, 카드한도는 평균~높다 -> 소득 적고, 카드 사용량 적지만 한도는 높음
- Cluster1: balance 높은편, 한번에 조금씩, 할부 거의 안함, 현금인출 중간, 한도높음, 지불금액 큼, 현금인출 조금씩 자주 -> 소득 많고 한도가 높고 카드 사용량 높음, 지불금액도 크다
- Cluster2: balance 높은 편, 카드 사용량 많은 편, 현금인출액 다양, 할부 금액 다양하게 자주함, 자주는 아님,쇼핑자주, 지불금액 큼, 한도 매우높음
- Cluster3: balance 적은 편, 구매량 적은 편, 한번 구매할 때 사용하는 최대금액 적은 편, 할부 금액 중간 자주함, 현금인출 거의 사용x,한도는 중간, 지불금액 중간, 자주 구매하지 않는 편
- Cluster4: balance는 중간, 구매금액은 적음, 최대구매금액 다양한 편, 할부 거의 안함, 현금인출 적음~중간, 한도 높은편, 카드값은 별로 안나옴,금액 최소값 작음, 쇼핑빈도수 다양, 현금인출 거의안함 => 아껴씀
- Cluster5: balance 중간,구매금액도 중간, 최소구매금액 다양, 할부금액 다양, 현금인출금액 매우적음,한도높음, 지불금액 중간, 최소지불금액 다양, 구매자주,할부빈도수 다양, 현금인출 거의 안함
- Cluster6: balance 중간, 구매 금액 매우적음, 최소구매금액도 매우적음, 할부금액 매우작음, 현금인출금액 다양. 한도높음, 지불금액 적음~중간, 최소지불금액 매우적음, 구매빈도수 매우 낮음, 할부 빈도수 매우 적음, 현금인출 잘 안하는 편, 자가지불 적음? 


- cluster0: People with average to high credit limit who purchase not so much.
- cluster1: People in high-wage class, purchase frequently, pay as much as they earn. -> high spender
- cluster2: People in high-wage class, purchase in installment frequently, cash in advance -> high spender
- cluster3: People with low-wage, purchase not frequently, pay in average, credit in average
- cluster4: People with average income, purchase little, frequently
- Cluster5: People with average income, purchase in average, frequently, pay in average, high credit -> standard, need to target
- Cluster6: little spender with average-income

In [ ]:
# Visualization of Clusters
# using PCA to transform data to 2 dimensions for visualization

dist=1-cosine_similarity(x) # apply cosine distance for x

pca=PCA(2) # transform 17 dimensions to 2 dimensions
pca.fit(dist)
x_pca=pca.transform(dist)
x_pca.shape

In [ ]:
x_pca

In [ ]:
x, y=x_pca[:, 0], x_pca[:,1]

colors={0:'red',
       1: 'blue',
       2:'green',
       3:'yellow',
       4:'orange',
       5:'purple',
       6:'magenta'}

names={0:'cluster0',
      1:'cluster1',
      2:'cluster2',
      3:'cluster3',
      4:'cluster4',
      5:'cluster5',
      6:'cluster6'}

df=pd.DataFrame({'x':x, 'y':y, 'label':labels})
groups=df.groupby('label')



fig, ax=plt.subplots(figsize=(20,13))


for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=5,
           color=colors[name], label=names[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(axis='x',which='both', bottom='off', top='off', labelbottom='off')
    ax.tick_params(axis='y',which='both', bottom='off', top='off', labelbottom='off')
    
ax.legend()
ax.set_title("Customer Segmentation based on their credit card using behavior")
plt.show()

